In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score

torch.manual_seed(2)
np.random.seed(2)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/electric-motor-temperature/pmsm_temperature_data.csv')
df.head()

In [ ]:
col_list = df.columns.tolist()
profile_id = ['profile_id']
target_list = ['pm', 'torque', 'stator_yoke', 'stator_tooth', 'stator_winding']
feature_list = [col for col in col_list if col not in target_list and col not in profile_id]

In [ ]:
df.info()

**As profile id just indicates which measurement session the data belongs to, it can be treated as categorical variable, and also to make select data that is relevant to the session**

In [ ]:
df['profile_id'] = df.profile_id.astype('category')
df.profile_id.unique()

**Since all the variables/features are measured over time and are time dependant. The later analysis is done assuming the data to be time series data.**

**For this reason, the data is split into number of time sequences. The target value for a given sequence will be the value which is just after a given sequence.**

# Data preparation

In [ ]:
def build_sequences(features_df, target_df, sequence_length = 10):
    """Builds sequences from data and converts them into pytorch tensors
        sequence_length - represents the number of samples to be considered in a sequence
    """
    data_ = []
    target_ = []
    
    for i in range(int(features_df.shape[0]/sequence_length)):
        
        data = torch.from_numpy(features_df.iloc[i:i+sequence_length].values.T)
        target = torch.from_numpy(target_df.iloc[i+sequence_length+1].values.T)
        
        data_.append(data)
        target_.append(target)
        
    data = torch.stack(data_)
    target = torch.stack(target_)
    
    
    return data, target

In [ ]:
prof_ids = list(df.profile_id.unique())
print(prof_ids)

**Selecting a measurement profile to prepare the data**

In [ ]:
prof_id = 6

curr_df = df[df['profile_id'] == prof_id]

curr_df = curr_df.drop('profile_id', axis = 1)
columns = curr_df.columns.tolist()

In [ ]:
scaler = MinMaxScaler()

curr_df = pd.DataFrame(scaler.fit_transform(curr_df), columns= columns)
curr_df.head()

**Sequence length has to be selected prior to the model initialization**

**As the sampling is done at 2 Hz, the sequence size can be >= 2. But, keeping in view the importance of speed of operation as the values are needed for other operations or control, it should be as small as possible to reduce the delay.**

**Also, another point to be noted here is, if the sequence length is higher the prediction error is less. So there must be a trade off between the required error rate and speed (in this case the initial prediction).**

**The delay in responce will be only at the beginning and the later predictions will not have the delay if real time operation is considered.**

In [ ]:
sequence_length = 5

features = curr_df[feature_list]
target = curr_df[target_list][['pm']]

data, target = build_sequences(features, target, sequence_length=sequence_length)

In [ ]:
data.shape

**Dividing the generated sequences into training and testing set**

In [ ]:
# Test size the percentage of data to be used for testing
test_size = 0.05

indices = torch.randperm(data.shape[0])

train_indices = indices[:int(indices.shape[0] * (1-test_size))]
test_indices = indices[int(indices.shape[0] * (1-test_size)):]

X_train, y_train = data[train_indices], target[train_indices]
X_test, y_test = data[test_indices], target[test_indices]

**A Dataset class is needed for the data inorder to use the dataloader of Pytorch**

In [ ]:
class PMSMDataset(torch.utils.data.dataset.Dataset):
    """Dataset with Rotor Temperature as Target"""
    def __init__(self, data, target):
        
        self.data = data
        self.target = target
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return self.data[idx].float(), self.target[idx].float()

In [ ]:
batch_size = 10

pm_train_dataset = PMSMDataset(X_train, y_train)
pm_train_loader = torch.utils.data.dataloader.DataLoader(pm_train_dataset, batch_size= batch_size)

pm_test_dataset = PMSMDataset(X_test, y_test)
pm_test_loader = torch.utils.data.dataloader.DataLoader(pm_test_dataset, batch_size= 1)

**The network class**

In [ ]:
class Network(nn.Module):
    def __init__(self, sequence_length, n_features):
        super(Network, self).__init__()
        
        self.sequence_length = sequence_length
        self.n_features = n_features
        
        self.features = nn.Sequential(
                        nn.Conv1d(n_features, 16, kernel_size=3),
                        nn.ReLU(),
                        nn.Conv1d(16,32, kernel_size=1))
        
        self.lin_in_size = self.get_lin_in_size()
        
        self.predictior = nn.Sequential(
                        nn.Linear(self.lin_in_size,30),
                        nn.ReLU(),
                        nn.Linear(30, 1))
        
    def forward(self, x):
        
        x = self.features(x)
        x = x.view(-1, self.lin_in_size)
        x = self.predictior(x)
        return x
    
    def get_lin_in_size(self):
        rand_in = torch.rand(10, self.n_features, self.sequence_length)
        rand_out = self.features(rand_in)
        return rand_out.shape[-1] * rand_out.shape[-2]
        

**Note that the same sequence length used to generate the data should be used to create the network**

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

n_features = X_train.shape[-2]
net = Network(sequence_length, n_features)
net = net.to(device)

In [ ]:
lr = 0.001

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

**Training**

In [ ]:
def test(net, test_loader, criterion):
    
    net.eval()
    losses = []
    for i, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device) 
        out = net(data)
        loss = criterion(out, target)
        losses.append(loss.item())  
    
    net.train()
    return np.mean(losses)

def train(n_epochs, net, train_loader, test_loader, optimizer, criterion, interval = 10):
    net.train()
    training_losses = []
    test_losses = []
    
    for epoch in range(1,n_epochs+1):
        running_loss = 0.0
        batch_losses = []
        
        for i, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device) 
            
            optimizer.zero_grad()
            out = net(data)
            loss = criterion(out, target)
            batch_losses.append(loss.item())
            
            loss.backward()
            optimizer.step()
        
        training_losses.append(np.mean(batch_losses))
        
        test_losses.append(test(net, test_loader, criterion))
        
        if epoch%interval==1:
            print("Epoch {}, Training loss {:.6f}, Testing loss {:.6f}".format(epoch, training_losses[-1], test_losses[-1]))
    return training_losses, test_losses

In [ ]:
training_losses, test_losses = train(50, net, pm_train_loader, pm_test_loader, optimizer, criterion)

In [ ]:
plt.plot(training_losses, label='Train')
plt.plot(test_losses, label ='Test')
plt.title("Loss vs Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

**Saving the model**

In [ ]:
torch.save(net.state_dict(), './model_single_measurement.pt')

**Testing**

In [ ]:
net = Network(sequence_length,n_features)
net.load_state_dict(torch.load('./model_single_measurement.pt'))
net = net.to(device)

In [ ]:
def get_predictions(net, loader):
    
    net.eval()
    targets = []
    outputs = []
    
    for _, (data, target) in enumerate(loader):
        
        data, target = data.to(device), target.to(device)
        
        out = net(data)

        targets.append(target.item())
        outputs.append(out.item())
    return targets, outputs

In [ ]:
targets, outputs = get_predictions(net, pm_test_loader)

plt.figure(figsize=(5,5))
plt.scatter(targets, outputs, s=10)
plt.xlabel("Target")
plt.ylabel("Predicted")
plt.title("Predicted vs Target")
plt.show()

In [ ]:
print("R2 Score - ", r2_score(targets, outputs))

**As seen in the plot the predicted value is almost equal to the target value, since the plot is a straight line. R2 Score also confirms this as it is close to 1.**

**The predictions are based on one measurement session, the next step is to train a single model on multiple measurement profiles and evaluate the performance.**